In [1]:
import pandas as pd
import numpy as np
import lightgbm
from function import *

In [3]:
train_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/Data_/train_data.csv")
test_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/Data_/test_data.csv")

In [4]:
train_df, test_df = finalize_data(train_df, test_df)

In [6]:
from sklearn.model_selection import KFold

In [11]:
X_cols = train_df.drop("tm", axis = 1).columns

In [12]:
kf = KFold(n_splits=5)
for train_index, val_index in kf.split(train_df[X_cols]):
    X_train = train_df[X_cols].iloc[train_index]
    X_test = train_df[X_cols].iloc[val_index]
    y_train = train_df["tm"].iloc[train_index]
    y_test = train_df["tm"].iloc[val_index]


In [13]:
lgb_train = lightgbm.Dataset(X_train, y_train)
lgb_test = lightgbm.Dataset(X_test, y_test)

In [16]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [17]:
gbm = lightgbm.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_test,
                callbacks=[lightgbm.early_stopping(stopping_rounds=5)])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007324 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's l2: 144.754	valid_0's l1: 8.68187
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 140.632	valid_0's l1: 8.56982
[3]	valid_0's l2: 136.944	valid_0's l1: 8.47078
[4]	valid_0's l2: 133.336	valid_0's l1: 8.37357
[5]	valid_0's l2: 130.482	valid_0's l1: 8.29125
[6]	valid_0's l2: 127.853	valid_0's l1: 8.2069
[7]	valid_0's l2: 125.621	valid_0's l1: 8.14337
[8]	valid_0's l2: 123.276	valid_0's l1: 8.07012
[9]	valid_0's l2: 121.063	valid_0's l1: 8.00285
[10]	valid_0's l2: 118.912	valid_0's l1: 7.93639
[11]	valid_0's l2: 117.189	valid_0's l1: 7.88095
[12]	valid_0's l2: 115.513	valid_0's l1: 7.82675
[13]	valid_0's l2: 113.826	valid_0's l1: 7.77846
[14]	valid_0's l2: 112.317	valid_0's l1: 7.73162
[15]	valid_0's l2: 111.042	valid_0's l1: 7.68598
[16]	valid_0's l2: 109.651	valid_0's l1: 7.64096
[

In [19]:
preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [20]:
score(preds, y_test)

SpearmanrResult(correlation=0.30376061841720137, pvalue=5.4573627223337295e-124)


In [21]:
preds_test = gbm.predict(test_df, num_iteration=gbm.best_iteration)

In [23]:
make_submmission(preds = preds_test, name = "LGBMtrainAPI.csv")